In [23]:
import numpy as np 
import pandas as pd 
import os 
import inventorize as inv
path = os.getcwd()
print(path)

/Users/dang/Desktop/Learning/Mygit/datascience_supplychain_practice/Jupyternotebook_source/Section_18_seasonal_inventory


In [7]:
retail_clean = pd.read_csv("{0}/retail_clean.csv".format(path))

In [8]:
retail_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779495 entries, 0 to 779494
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   779495 non-null  int64  
 1   Invoice      779495 non-null  int64  
 2   StockCode    779495 non-null  object 
 3   Description  779495 non-null  object 
 4   Quantity     779495 non-null  int64  
 5   InvoiceDate  779495 non-null  object 
 6   Price        779495 non-null  float64
 7   Customer ID  779495 non-null  float64
 8   Country      779495 non-null  object 
 9   Revenue      779495 non-null  float64
dtypes: float64(3), int64(3), object(4)
memory usage: 59.5+ MB


In [9]:
retail_clean['InvoiceDate'] = pd.to_datetime(retail_clean['InvoiceDate'])

In [10]:
retail_clean['date'] = retail_clean['InvoiceDate'].dt.strftime("%Y-%m-%d")

In [13]:
retail_clean['date'] = pd.to_datetime(retail_clean['date'])

In [14]:
retail_clean['year'] = retail_clean['date'].dt.year

In [15]:
years_2 = retail_clean[retail_clean.year.isin([2010,2011])]

In [16]:
total = years_2.groupby(['year', 'Description']).agg(
    total_sales=('Quantity', np.sum),
    price = ('Price', 'mean')
).reset_index()

In [17]:
expected = total.groupby('Description').agg(
    expected_Demand = ('total_sales', np.mean),
    sd = ('total_sales', 'std'),
    price = ('price', np.mean)
    ).reset_index()

In [18]:
expected['sd']

0              NaN
1              NaN
2        53.740115
3              NaN
4              NaN
           ...    
5146           NaN
5147     84.852814
5148    545.886435
5149           NaN
5150           NaN
Name: sd, Length: 5151, dtype: float64

In [19]:
def margin_error(dataframe):
    if(pd.isna(dataframe['sd'])):
        a = dataframe['expected_Demand']*0.1
    else:
        a = dataframe['sd']
    return a

In [20]:
expected['sd1'] = expected.apply(margin_error, axis=1)

In [21]:
expected['cost'] = expected['price'] * 0.4

In [22]:
empty_data = pd.DataFrame()

In [25]:
for i in range(expected.shape[0]):
    a = inv.MPN_singleperiod(
        expected.loc[i,'expected_Demand'],
        expected.loc[i, 'sd1'],
        expected.loc[i, 'price'],
        expected.loc[i, 'cost'],
        0, 0
    )
    b = pd.DataFrame(a, index=[0])
    b['description'] = expected.loc[i, 'Description']
    empty_data = pd.concat([empty_data, b], axis=0)

/Users/dang/Desktop/Learning/Mygit/datascience_supplychain_practice/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2098: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/dang/Desktop/Learning/Mygit/datascience_supplychain_practice/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2099: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [27]:
empty_data

,quantity,demand,sd,unitshort,shortagecost,cost,revenue,profit,soldatfullprice,sold_discount,description
0,171.230897,167.0,16.700000,4.759562,31.604423,454.802691,1108.912745,622.505631,162.240438,8.990458,DOORMAT UNION JACK GUNS AND ROSES
0,409.108549,399.0,39.900000,11.371647,22.077241,317.702011,774.630000,434.850748,387.628353,21.480197,3 STRIPEY MICE FELTCRAFT
0,175.614903,162.0,53.740115,15.316131,34.067452,156.247088,360.334286,170.019746,146.683869,28.931034,4 PURPLE FLOCK DINNER CANDLES
0,1932.755929,1885.0,188.500000,53.723196,67.050492,964.888501,2352.618349,1320.679356,1831.276804,101.479125,50'S CHRISTMAS GIFT BAG LARGE
0,173.281566,169.0,16.900000,4.816562,1.011478,14.555652,35.490000,19.922870,164.183438,9.098128,ANIMAL STICKERS
...,...,...,...,...,...,...,...,...,...,...,...
0,5017.988072,4894.0,489.400000,139.480807,116.741919,1679.971805,4096.154622,2299.440898,4754.519193,263.468879,ZINC T-LIGHT HOLDER STARS SMALL
0,90.497215,69.0,84.852814,24.183365,356.791006,534.062848,1017.996429,127.142575,44.816635,45.680580,ZINC TOP 2 DOOR WOODEN SHELF
0,2956.298747,2818.0,545.886435,155.579650,133.888613,1017.651706,2425.112237,1273.571918,2662.420350,293.878397,ZINC WILLIE WINKIE CANDLE STICK
0,25.633368,25.0,2.500000,0.712509,5.112254,73.567765,179.375000,100.694981,24.287491,1.345877,ZINC WIRE KITCHEN ORGANISER


In [28]:
empty_data.iloc[1,:]

quantity                          409.108549
demand                            399.000000
sd                                 39.900000
unitshort                          11.371647
shortagecost                       22.077241
cost                              317.702011
revenue                           774.630000
profit                            434.850748
soldatfullprice                   387.628353
sold_discount                      21.480197
description         3 STRIPEY MICE FELTCRAFT
Name: 0, dtype: object